In [ ]:
import pandas as pd

df = pd.read_csv("data/citibike-tripdata.csv")
df.info()
display(df.head())

In [3]:
# Сколько пропусков в столбце start station id?
missing_start_station_id = df['start station id'].isnull().sum()
display(missing_start_station_id)

169

In [4]:
# Какой тип данных имеют столбцы starttime и stoptime?
print("Тип данных для столбца 'starttime':", df['starttime'].dtype)
print("Тип данных для столбца 'stoptime':", df['stoptime'].dtype)

Тип данных для столбца 'starttime': object
Тип данных для столбца 'stoptime': object


In [5]:
# Найдите идентификатор самой популярной стартовой стоянки. Запишите идентификатор в виде целого числа.
most_popular_start_station_id = df['start station id'].value_counts().idxmax()
display(most_popular_start_station_id)

281.0

In [7]:
# Велосипед с каким идентификатором является самым популярным?
most_popular_bike_id = df['bikeid'].value_counts().idxmax()
display(most_popular_bike_id)

33887

In [8]:
# Какой тип клиентов (столбец usertype) является преобладающим — Subscriber или Customer? 
# В качестве ответа запишите долю клиентов преобладающего типа среди общего количества клиентов. Ответ округлите до сотых.
# Посчитаем количество каждого типа клиентов
user_type_counts = df['usertype'].value_counts()

# Найдем преобладающий тип клиентов
dominant_user_type = user_type_counts.idxmax()

# Найдем долю преобладающего типа клиентов
dominant_user_type_percentage = user_type_counts[dominant_user_type] / len(df)
display(dominant_user_type_percentage)

0.7740066666666666

In [10]:
# Кто больше занимается велоспортом — мужчины или женщины? 
# В ответ запишите число поездок для той группы, у которой их больше.

# Посчитаем количество поездок для каждого значения gender
gender_counts = df['gender'].value_counts()

# Найдем гендер с наибольшим количеством поездок
dominant_gender = gender_counts.idxmax()

# Найдем количество поездок для гендера с наибольшим количеством
dominant_gender_trips = gender_counts[dominant_gender]
display(dominant_gender_trips)

183582

In [11]:
# A. Проверим, что число уникальных стартовых и конечных стоянок не равны друг другу
unique_start_stations = df['start station id'].nunique()
unique_end_stations = df['end station id'].nunique()
check_a = unique_start_stations != unique_end_stations

# B. Проверим, что минимальный возраст клиента составлял 10 лет
current_year = pd.Timestamp.now().year
min_birth_year = df['birth year'].min()
min_age = current_year - min_birth_year
check_b = min_age == 10

# C. Проверим, что самой непопулярной стартовой стоянкой является "Eastern Pkwy & Washington Ave"
unpopular_start_station = "Eastern Pkwy & Washington Ave"
count_unpopular_start_station = df[df['start station name'] == unpopular_start_station].shape[0]
check_c = count_unpopular_start_station == df['start station name'].value_counts().min()

# D. Проверим, что наибольшее количество поездок завершается на стоянке "Liberty Light Rail"
popular_end_station = "Liberty Light Rail"
count_popular_end_station = df[df['end station name'] == popular_end_station].shape[0]
check_d = count_popular_end_station == df['end station name'].value_counts().max()

# Выведем результаты
print("A. Число уникальных стартовых и конечных стоянок не равны:", check_a)
print("B. Минимальный возраст клиента составлял 10 лет:", check_b)
print("C. Самой непопулярной стартовой стоянкой является 'Eastern Pkwy & Washington Ave':", check_c)
print("D. Наибольшее количество поездок завершается на стоянке 'Liberty Light Rail':", check_d)

A. Число уникальных стартовых и конечных стоянок не равны: True
B. Минимальный возраст клиента составлял 10 лет: False
C. Самой непопулярной стартовой стоянкой является 'Eastern Pkwy & Washington Ave': True
D. Наибольшее количество поездок завершается на стоянке 'Liberty Light Rail': False


In [12]:
# Удалим столбцы с идентификаторами стоянок
df_cleaned = df.drop(['start station id', 'end station id'], axis=1)

# Выведем количество оставшихся столбцов
remaining_columns = df_cleaned.shape[1]
display(remaining_columns)

12

In [13]:
# Замените признак birth year на более понятный признак возраста клиента age. Годом отсчёта возраста выберите 2018 год. 
# Столбец birth year удалите из таблицы. Сколько поездок совершено клиентами старше 60 лет?
# Заменим признак birth year на признак возраста age
current_year = 2018
df['age'] = current_year - df['birth year']

# Удалим столбец birth year
df = df.drop(['birth year'], axis=1)

# Посчитаем количество поездок клиентов старше 60 лет
trips_by_clients_over_60 = df[df['age'] > 60].shape[0]
display(trips_by_clients_over_60)

11837

In [14]:
# Создайте признак длительности поездки trip duration. Для этого вычислите интервал времени между временем окончания поездки
# (stoptime) и её началом (starttime). Сколько целых минут длилась поездка под индексом 3 в таблице?
# Преобразуем столбцы starttime и stoptime в формат datetime
df['starttime'] = pd.to_datetime(df['starttime'])
df['stoptime'] = pd.to_datetime(df['stoptime'])

# Создадим столбец trip duration
df['trip duration'] = (df['stoptime'] - df['starttime']).dt.total_seconds() / 60  # переводим в минуты

# Найдем длительность поездки под индексом 3
trip_duration_index_3 = df.loc[3, 'trip duration']
display(trip_duration_index_3)

7.280616666666666

In [15]:
# Создайте «признак-мигалку» weekend, который равен 1, если поездка начиналась в выходной день (суббота или воскресенье), 
# и 0 — в противном случае. Выясните, сколько поездок начиналось в выходные.
# Преобразуем столбец starttime в формат datetime
df['starttime'] = pd.to_datetime(df['starttime'])

# Создадим признак-мигалку weekend
df['weekend'] = (df['starttime'].dt.dayofweek // 5).astype(int)  # 5 и 6 соответствуют субботе и воскресенью

# Посчитаем количество поездок, начинающихся в выходные
weekend_trips = df[df['weekend'] == 1].shape[0]
display(weekend_trips)

115135

In [16]:
# Создайте признак времени суток поездки time_of_day. Время суток будем определять из часа начала поездки
# Преобразуем столбец starttime в формат datetime
df['starttime'] = pd.to_datetime(df['starttime'])

# Создадим признак времени суток time_of_day
df['time_of_day'] = pd.cut(df['starttime'].dt.hour, bins=[0, 6, 12, 18, 23], labels=['night', 'morning', 'day', 'evening'], include_lowest=True)

# Посчитаем количество поездок, совершённых днём и ночью
day_trips = df[df['time_of_day'] == 'day'].shape[0]
night_trips = df[df['time_of_day'] == 'night'].shape[0]

# Выведем результат
if night_trips != 0:
    ratio = day_trips / night_trips
    print("Количество поездок, совершённых днём, в", round(ratio), "раз(а) больше, чем количество поездок, совершённых ночью.")
else:
    print("Количество поездок, совершённых ночью, равно нулю.")

Количество поездок, совершённых днём, в 9 раз(а) больше, чем количество поездок, совершённых ночью.
